#1. 데이터 읽기

In [ ]:
import pandas as pd 
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()
df = pd.DataFrame(data=cancer.data, columns=cancer.feature_names) 
df["class"] = cancer.target 
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,class
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


#2. 모델 만들기
> voting_classifier(분류기 리스트, voting="보팅기법")으로 선언해주어야 한다.

In [ ]:
from sklearn.linear_model import LogisticRegression 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import VotingClassifier 
from sklearn.metrics import accuracy_score 

# 가장 마지막에 label이 저장되어 있어 -1로 표현함.
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:-1], df.iloc[:,-1],test_size=0.2) 
# 로지스틱 회귀
lr = LogisticRegression(max_iter=4000) 
# KNN
knn = KNeighborsClassifier() 
vo = VotingClassifier([("lr", lr), ("knn", knn)], voting="soft") 
vo.fit(X_train, y_train) 
y_pred = vo.predict(X_test) 

print("정확도 : {}\n\n".format(accuracy_score(y_test, y_pred))) 

정확도 : 0.9649122807017544




In [ ]:
for clf in [lr, knn]: 
  print('클래스 정보 확인 ',clf.__class__)
  clf_name = clf.__class__.__name__
  clf.fit(X_train, y_train) 
  y_pred = clf.predict(X_test) 
  acc = accuracy_score(y_test, y_pred) 
  print("{}'s 정확도 : {}'".format(clf_name,acc))

클래스 정보 확인  <class 'sklearn.linear_model._logistic.LogisticRegression'>
LogisticRegression's 정확도 : 0.9824561403508771'
클래스 정보 확인  <class 'sklearn.neighbors._classification.KNeighborsClassifier'>
KNeighborsClassifier's 정확도 : 0.9385964912280702'


> 다음과 같은 경고가 나타날 경우 lr의 max_iter의 수치를 올린다.

>Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html

>Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)

#3. 스케일링을 통한 모델 성능 높이기
> 로지스틱 회귀의 경우 데이터를 스케일링하게 되면 성능이 향상됨으로 StandardScaler를 통해 스케일링을 진행해 보자

In [ ]:
from sklearn.preprocessing import StandardScaler 

X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,:-1], df.iloc[:,-1],test_size=0.2, random_state=62) 
lr = LogisticRegression(max_iter=4000) 
knn = KNeighborsClassifier() 
vo = VotingClassifier([("lr", lr), ("knn", knn)], voting="soft") 
vo.fit(X_train, y_train) 
y_pred = vo.predict(X_test) 

print("정확도 : {}\n\n".format(accuracy_score(y_test, y_pred))) 

for clf in [lr, knn]: 
  print('클래스 정보 확인 ',clf.__class__)
  clf_name = clf.__class__.__name__
  clf.fit(X_train, y_train) 
  y_pred = clf.predict(X_test) 
  acc = accuracy_score(y_test, y_pred) 
  print("{}'s 정확도 : {}'".format(clf_name,acc))
print('===============================================================')


scaler = StandardScaler() 
X_train = scaler.fit_transform(X_train) 

scaler = StandardScaler() 
X_test = scaler.fit_transform(X_test) 

vo = VotingClassifier([("lr", lr), ("knn", knn)], voting="soft") 

vo.fit(X_train, y_train) 
y_pred = vo.predict(X_test) 

print("정확도 : {}".format(accuracy_score(y_test, y_pred))) 

for clf in [lr, knn]: 
  clf_name = clf.__class__.__name__ 
  clf.fit(X_train, y_train) 
  y_pred = clf.predict(X_test) 
  acc = accuracy_score(y_test, y_pred) 
  print("{}'s acc : {}'".format(clf_name,acc))

정확도 : 0.9298245614035088


클래스 정보 확인  <class 'sklearn.linear_model._logistic.LogisticRegression'>
LogisticRegression's 정확도 : 0.9298245614035088'
클래스 정보 확인  <class 'sklearn.neighbors._classification.KNeighborsClassifier'>
KNeighborsClassifier's 정확도 : 0.9122807017543859'
정확도 : 0.9736842105263158
LogisticRegression's acc : 0.9824561403508771'
KNeighborsClassifier's acc : 0.956140350877193'


#4. 교차검증

In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_score 

for clf in [vo, lr, knn]:
  # 위에서 표준화 작업을 진행하여 두 값을 합치기 위해 concatenate을 이용한다.
  scores = cross_val_score(clf, np.concatenate((X_train,X_test)), 
                           np.concatenate((y_train, y_test)), 
                           scoring="accuracy", cv=5) 
  print(clf.__class__.__name__) 
  print(scores) 
  print(np.mean(scores)) 
  print()

VotingClassifier
[0.96491228 0.99122807 0.99122807 0.98245614 0.97345133]
0.9806551777674274

LogisticRegression
[0.94736842 0.97368421 0.99122807 0.99122807 0.98230088]
0.9771619313771154

KNeighborsClassifier
[0.95614035 0.96491228 0.97368421 0.94736842 0.95575221]
0.9595714951094549



#5. 튜닝
> 하이퍼 파라미터 설정 시 이전 VotingClassifier에서 설정한 쌍따옴표 안의 문자와 언더바(_) 두 개를 이어서 파라미터를 설정할 수 있다.

In [ ]:
from sklearn.model_selection import GridSearchCV

def get_best_hyper_paramerter(logreg_list, n_neighbors_list):
  params =  {
    'lr__C' : logreg_list,
    'knn__n_neighbors' : n_neighbors_list,
    'knn__weights' : ["uniform", "distance"],
    'knn__metric' : ['euclidean', 'manhattan', 'minkowski']
  }

  grid_cv = GridSearchCV(vo, param_grid=params, scoring='accuracy', cv=5)
  grid_cv.fit(X_train, y_train)
  print('Best Score: ', grid_cv.best_score_)
  print('Best Hyper Parameter: ', grid_cv.best_params_)

> 파라미터는 알고리즘마다 다르게 나타나며 각 알고리즘에 맞는 파라미터를 지정하기 위해 3번에서 VotingClassifier에 적용된 값을 사용하여 lr__C 형식으로 만들게 된다.

In [ ]:
cost_list =  [0.001, 0.01, 0.1, 1, 10]
n_neighbors_list =  range(1, 90, 10)

get_best_hyper_paramerter(cost_list, n_neighbors_list)

Best Score:  0.9758241758241759
Best Hyper Parameter:  {'knn__metric': 'euclidean', 'knn__n_neighbors': 61, 'knn__weights': 'distance', 'lr__C': 10}


In [ ]:
cost_list =  range(2, 22, 4)
n_neighbors_list =  range(52, 70, 3)

get_best_hyper_paramerter(cost_list, n_neighbors_list)

Best Score:  0.9758241758241759
Best Hyper Parameter:  {'knn__metric': 'euclidean', 'knn__n_neighbors': 58, 'knn__weights': 'uniform', 'lr__C': 10}


In [ ]:
cost_list =  range(7, 13, 1)
n_neighbors_list =  range(56, 60, 1)

get_best_hyper_paramerter(cost_list, n_neighbors_list)

Best Score:  0.9758241758241759
Best Hyper Parameter:  {'knn__metric': 'euclidean', 'knn__n_neighbors': 56, 'knn__weights': 'distance', 'lr__C': 10}
